In [2]:
!pip install fabric-data-agent-sdk

INFO: pip is looking at multiple versions of azure-kusto-data to determine which version is compatible with other requirements. This could take a while.
Reason for being yanked: Yanked due to conflicts with CVE-2024-35195 mitigation
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 785.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.3/699.3 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 956.3/956.3 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.0/349.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from fabric.dataagent.client import(
    FabricDataAgentManagement,
    create_data_agent,
    delete_data_agent,
)

In [4]:
#Pretty printing helper
def pretty_print(messages):
    print("Message")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [5]:
data_agent_name = "SemDoc_agent"
#Create Data Agent
data_agent = create_data_agent(data_agent_name)
#or initiate existing agent
data_agent = FabricDataAgentManagement(data_agent_name)

In [6]:
data_agent.update_configuration(
    instructions = "You are a Power BI semantic model developer. \
The adventureworksdw2020_calcdepency table contains the output of the DAX INFO.CALCDEPENDENCY() function for another Power BI semantic model. It contains information about the dependencies between the columns, tables, measures, calculated columns, calculated tables and relationships in that model. \
Each row in the CalcDependencies table represents a dependency between two objects. \
The combination of values in the OBJECT_TYPE, TABLE and OBJECT column uniquely identifies the object which is the source of the dependency. \
The combination of values in the REFERENCED_OBJECT_TYPE, REFERENCED_TABLE and _REFERENCED_OBJECT columns uniquely identifies the object which is the target of the identity. \
You can join the table to itself multiple times to find chains of dependencies between objects. \
When I ask about dependencies, please include direct dependencies and also indirect dependencies, for example where one object has a dependency on another object which in turn has a dependency on the object I am asking about.\
The EXPRESSION column contains the DAX definition of an object. \
A value of CALC_COLUMN in the OBJECT_TYPE table indicates that the object is a calculated column.",
    #user_description = "Data agent to  automate semantic model documentation",  
)

data_agent.get_configuration()

DataAgentConfiguration(instructions='You are a Power BI semantic model developer. The adventureworksdw2020_calcdepency table contains the output of the DAX INFO.CALCDEPENDENCY() function for another Power BI semantic model. It contains information about the dependencies between the columns, tables, measures, calculated columns, calculated tables and relationships in that model. Each row in the CalcDependencies table represents a dependency between two objects. The combination of values in the OBJECT_TYPE, TABLE and OBJECT column uniquely identifies the object which is the source of the dependency. The combination of values in the REFERENCED_OBJECT_TYPE, REFERENCED_TABLE and _REFERENCED_OBJECT columns uniquely identifies the object which is the target of the identity. You can join the table to itself multiple times to find chains of dependencies between objects. When I ask about dependencies, please include direct dependencies and also indirect dependencies, for example where one object

In [7]:
#add a lakehouse
lakehouse_name = "Documentaion"
data_agent.add_datasource(lakehouse_name,type ="lakehouse")



Datasource(0cb93417-2010-4851-8d73-082ee9d89b32)

In [8]:
#Retrieve the data source object (assume one was added)
datasource = data_agent.get_datasources()[0]

#Select relevant tables from the Lakehouse (schema: dbo)
datasource_table = "adventureworksdw2020_calcdepency"
datasource.select("dbo", datasource_table)

In [9]:
datasource.pretty_print()

 dbo
  | adventureworksdw2020_calcdepency *
  |  | DATABASE_NAME
  |  | OBJECT_TYPE
  |  | TABLE
  |  | OBJECT
  |  | EXPRESSION
  |  | REFERENCED_OBJECT_TYPE
  |  | REFERENCED_TABLE
  |  | REFERENCED_OBJECT
  |  | REFERENCED_EXPRESSION
  |  | QUERY


'Documentaion'

In [10]:
data_agent.publish()

In [11]:
import sempy.fabric as fabric
from fabric.dataagent.client import FabricOpenAI

fabric_client = FabricOpenAI(artifact_name = data_agent_name)
assistant = fabric_client.beta.assistants.create(model= "gpt-4o")
thread = fabric_client.beta.threads.create()

print(assistant.id)
print(thread.id)

/tmp/ipykernel_42/239863682.py:6: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = fabric_client.beta.threads.create()


In [12]:
fabric_client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "Generate some documentation for the measures listed in the adventureworksdw2020_calcdepency table. Describe what each calculation does and how it works.",
)

run = fabric_client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id 
)

/tmp/ipykernel_42/2726624726.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  fabric_client.beta.threads.messages.create(
/tmp/ipykernel_42/2726624726.py:7: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = fabric_client.beta.threads.runs.create(


In [13]:
response = fabric_client.beta.threads.messages.list(thread_id = thread.id, order ="asc")
pretty_print(response)

/tmp/ipykernel_42/812547712.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  response = fabric_client.beta.threads.messages.list(thread_id = thread.id, order ="asc")


In [19]:
from pyspark.sql import SparkSession

# Spark session oluştur
spark = SparkSession.builder \
    .appName("MarkdownExport") \
    .getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/02 14:27:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/02 14:27:13 WARN SparkContext: Exception when load sparklyr connector java.lang.ClassNotFoundException: org.apache.spark.sparklyr.DefaultConnector


In [26]:
messages = list(response)
last_message = messages[-1]
#print(last_message)

#Create a single-row DataFrame with the markdown content
#df = spark.createDataFrame([(last_message.content[0].text.value,)],["markdown"])
df = spark.createDataFrame([(last_message.content[0].text.value,)],["text"])
df2 = df.toPandas()

#File name and path
datasource_table = "adventureworksdw2020_calcdepency"
dir_path = "abfss://e926f959-aad5-4096-beab-1f44d8bbec66@onelake.dfs.fabric.microsoft.com/0cb93417-2010-4851-8d73-082ee9d89b32/Files/model_documentation"
file_name = f"{dir_path}/{datasource_table}_documentation.md"

df2.to_csv(file_name, sep='\t', index=False)


In [25]:
import pandas as pd

# Son mesajın text içeriğini al
messages = list(response)
last_message = messages[-1]
text_content = last_message.content[0].text.value

# Pandas DataFrame'e koy (tek satır, tek sütun)
df = pd.DataFrame([[text_content]], columns=["text"])

# Dosya adı ve path
datasource_table = "adventureworksdw2020_calcdepency"
dir_path = "abfss://e926f959-aad5-4096-beab-1f44d8bbec66@onelake.dfs.fabric.microsoft.com/0cb93417-2010-4851-8d73-082ee9d89b32/Files/model_documentation"
file_name = f"{dir_path}/{datasource_table}_documentation.md"

# Kaydet (Markdown dosyası gibi)
df.to_csv(file_name, sep='\t', index=False)


In [1]:
!git

usage: git [-v | --version] [-h | --help] [-C <path>] [-c <name>=<value>]
           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]
           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]
           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]
           [--config-env=<name>=<envvar>] <command> [<args>]

These are common Git commands used in various situations:

start a working area (see also: git help tutorial)
   clone     Clone a repository into a new directory
   init      Create an empty Git repository or reinitialize an existing one

work on the current change (see also: git help everyday)
   add       Add file contents to the index
   mv        Move or rename a file, a directory, or a symlink
   restore   Restore working tree files
   rm        Remove files from the working tree and from the index

examine the history and state (see also: git help revisions)
   bisect    Use binary search to find the commit that introduced 

In [3]:
import pandas as pd

# Fabric içindeki dosyayı oku
df = pd.read_csv("abfss://e926f959-aad5-4096-beab-1f44d8bbec66@onelake.dfs.fabric.microsoft.com/0cb93417-2010-4851-8d73-082ee9d89b32/Files/model_documentation/adventureworksdw2020_calcdepency_documentation.md", sep="\t")

# Lokal path’e kaydet
df.to_csv("/tmp/adventureworksdw2020_calcdepency_documentation.md", sep="\t", index=False)
